# Test for Fixed Effects

## Load Libraries and Dataset

### Libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS

### Dataset and PanelOLS Object

In [2]:
sgx = pd.read_csv("data/clean_sgx.csv", index_col= None)
sgx = sgx[['Company Code', 'Year', 'LEVERAGE',
           'SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV']]
sgx['year'] = pd.Categorical(sgx['Year'])
sgx['compcode'] = pd.Categorical(sgx['Company Code'])
sgx = sgx.set_index(['Company Code', 'Year'])

endo = sgx.LEVERAGE
exog_vars = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV']
exog = sm.add_constant(sgx[exog_vars])

pooled_mod = PooledOLS(endo, exog)
pooled_res = pooled_mod.fit()

In [13]:
sgx.head(30)


LEVERAGE       SIZE  PROFITABILITY      TANG    LIQUID  \
Company Code Year                                                             
3            2015.0 -0.235443  17.927564      -0.094659  0.176726  1.563892   
             2016.0 -0.206973  17.984885      -0.010839  0.096761  1.505202   
             2017.0 -0.238219  18.136273      -0.013888  0.081307  1.343012   
             2018.0 -0.259016  18.114074       0.013371  0.079477  1.351929   
             2019.0 -0.222870  17.978821       0.078838  0.113450  1.558678   
             2020.0 -0.353373  18.184655       0.126747  0.077289  1.653512   
             2021.0 -0.354726  18.488321       0.118694  0.060646  1.582983   
             2022.0 -0.473794  18.382137       0.079766  0.063917  1.825263   
10           2015.0 -0.027551  17.648089       0.029971  0.094432  1.726605   
             2016.0  0.026441  17.779968       0.035823  0.151546  1.365213   
             2017.0  0.071474  17.879947       0.042540  0.163954  1.398815   
             2018.0  0.124074  17.825039      -0.001821  0.153358  1.303544   
             2019.0  0.267393  17.789708      -0.071748  0.298266  1.079697   
             2020.0  0.300076  17.569981       0.006588  0.283695  1.066758   
             2021.0  0.231850  17.314345      -0.032096  0.229591  1.047314   
             2022.0  0.234610  17.319097       0.068364  0.187101  1.020956   
12           2015.0  0.199280  16.916278      -0.037227  0.042269  0.536030   
             2016.0  0.073882  16.805376      -0.034099  0.039883  0.922022   
             2017.0  0.110147  16.475071      -0.230511  0.041917  1.021277   
             2018.0  0.215825  16.442061      -0.150514  0.023723  0.591639   
             2019.0  0.282343  16.787616       0.173040  0.056827  0.953543   
             2020.0  0.334869  16.820698      -0.154581  0.040020  0.921919   
             2021.0  0.336646  16.516268       0.065744  0.031697  0.539579   
             2022.0  0.256070  16.609800      -0.067886  0.033637  1.397857   
14           2015.0  0.226407  16.374310      -0.290508  0.111944  1.161707   
             2016.0  0.131923  16.313605      -0.108950  0.105254  1.003601   
             2017.0 -0.230777  16.558217      -0.167373  0.097032  1.170574   
             2018.0 -0.100940  16.666088      -0.105473  0.291681  0.815182   
             2019.0 -0.024855  16.548582      -0.177847  0.346863  0.558211   
             2020.0  0.019507  16.513552      -0.060864  0.394418  0.477831   

                             MCAP     SOLV    year compcode  
Company Code Year                                            
3            2015.0  3.726956e+07  -31.451  2015.0        3  
             2016.0  6.995037e+06   -7.719  2016.0        3  
             2017.0  7.919165e+06   -8.678  2017.0        3  
             2018.0  9.212378e+06    0.966  2018.0        3  
             2019.0  1.931658e+07   11.465  2019.0        3  
             2020.0  1.459661e+07   61.729  2020.0        3  
             2021.0  2.725603e+07  194.000  2021.0        3  
             2022.0  4.331235e+07   56.147  2022.0        3  
10           2015.0  1.888834e+07   -1.277  2015.0       10  
             2016.0  8.499886e+06    1.285  2016.0       10  
             2017.0  9.514661e+06    0.764  2017.0       10  
             2018.0  6.367345e+06   -2.850  2018.0       10  
             2019.0  3.209531e+06   -5.687  2019.0       10  
             2020.0  2.340626e+06   -3.112  2020.0       10  
             2021.0  1.254450e+06   -5.483  2021.0       10  
             2022.0  1.134966e+06   -0.086  2022.0       10  
12           2015.0  3.690988e+07   -2.039  2015.0       12  
             2016.0  4.643639e+07   -3.355  2016.0       12  
             2017.0  5.263948e+07  -24.447  2017.0       12  
             2018.0  3.172811e+07   -3.424  2018.0       12  
             2019.0  2.625733e+07    2.233  2019.0       12  
             2020.0  3.505423e+07   -2.289  2020.0       12  
    

## F-test for Fixed Effects (Entity-clustered Robust)

In [14]:
def fixedeffect(endo = endo, exog = exog,
                entity_eff = True, time_eff = True,
                cov_type = 'clustered', cluster_entity = True, **cov_kwargs):
    mod = PanelOLS(endo, exog,
                   entity_effects=entity_eff,
                   time_effects=time_eff)
    res = mod.fit(cov_type= cov_type, cluster_entity = cluster_entity, **cov_kwargs)

    return res

### Joint test for two-way FE

In [15]:
fixedeffect().summary

Dep. Variable:,LEVERAGE,R-squared:,0.2294
Estimator:,PanelOLS,R-squared (Between):,0.1387
No. Observations:,1728,R-squared (Within):,0.2397
Date:,"Mon, Apr 01 2024",R-squared (Overall):,0.1569
Time:,00:51:05,Log-likelihood,1628.0
Cov. Estimator:,Clustered,,
,,F-statistic:,74.386
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(6,1499)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,15.283


***Joint F-test for FE for eneity cluster on***

F-stat: **74.386**

F-stat (robust): **15.283**

### Conditional test for entity FE

In [16]:
fixedeffect(entity_eff=True, time_eff=False).summary

Dep. Variable:,LEVERAGE,R-squared:,0.2401
Estimator:,PanelOLS,R-squared (Between):,0.1308
No. Observations:,1728,R-squared (Within):,0.2401
Date:,"Mon, Apr 01 2024",R-squared (Overall):,0.1504
Time:,00:51:24,Log-likelihood,1615.1
Cov. Estimator:,Clustered,,
,,F-statistic:,79.286
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(6,1506)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,18.662


***Conditional F-test for entity-FE for eneity cluster on***

F-stat: **79.286**

F-stat (robust): **18.662**

### Conditional test for time FE

In [17]:
fixedeffect(entity_eff=False, time_eff=True).summary

Dep. Variable:,LEVERAGE,R-squared:,0.3616
Estimator:,PanelOLS,R-squared (Between):,0.4295
No. Observations:,1728,R-squared (Within):,0.0537
Date:,"Mon, Apr 01 2024",R-squared (Overall):,0.3621
Time:,00:52:08,Log-likelihood,285.77
Cov. Estimator:,Clustered,,
,,F-statistic:,161.80
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(6,1714)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,44.523


***Conditional F-test for time-FE for eneity cluster on***

F-stat: **161.80**

F-stat (robust): **44.523**

### Marginal test for entity FE

In [18]:
exog_vars_marginal_entity = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV', 'year']
exog_marginal_entity = sm.add_constant(sgx[exog_vars_marginal_entity])
fixedeffect(exog = exog_marginal_entity, time_eff = False).summary

Dep. Variable:,LEVERAGE,R-squared:,0.2513
Estimator:,PanelOLS,R-squared (Between):,0.1387
No. Observations:,1728,R-squared (Within):,0.2513
Date:,"Mon, Apr 01 2024",R-squared (Overall):,0.1589
Time:,00:56:23,Log-likelihood,1628.0
Cov. Estimator:,Clustered,,
,,F-statistic:,38.706
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(13,1499)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,13.731


***Marginal F-test for time-FE for eneity cluster on***

F-stat: **38.706**

F-stat (robust): **13.731**

### Marginal test for time FE

In [19]:
exog_vars_marginal_time = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV', 'compcode']
exog_marginal_time = sm.add_constant(sgx[exog_vars_marginal_time])
fixedeffect(exog = exog_marginal_time, entity_eff = False).summary

Dep. Variable:,LEVERAGE,R-squared:,0.8650
Estimator:,PanelOLS,R-squared (Between):,1.0000
No. Observations:,1728,R-squared (Within):,0.2397
Date:,"Mon, Apr 01 2024",R-squared (Overall):,0.8636
Time:,00:57:40,Log-likelihood,1628.0
Cov. Estimator:,Clustered,,
,,F-statistic:,43.449
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(221,1499)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,-1.21e+16


F-statistic = **43.449** for marginal entity effect.